# Analysis of Fortune's Top Companies of 2021

### Dataset - Fortune 1000

Every year Fortune, an American Business Magazine, publishes the Fortune 500, which ranks the top 500 corporations by revenue. 
However, the dataset used for this study includes all 1000 companies, as opposed to just the top 500.

The Fortune 1000 dataset used for this study has been taken from [here](https://www.kaggle.com/datasets/winston56/fortune-500-data-2021), and contains U.S. company data for the year 2021, with market valuations determined on January 20,2021.
The dataset has 1000 rows and 18 columns/attributes, namely:

|Attribute|Description|Note|Data Type|
|---------|-----------|----|-----|
|Company            | The name of the company| |object|
|Rank               | The ranking given to the company by Fortune in 2021| Values range from 1 (most revenue) to 1000 (least revenue)|object|
|Rank Change        | The change in the company's ranking from 2020 to 2021| Company can only have a change in rank if it is currently in the top 500 and was in the top 500 the year before|int64|
|Revenue            | Dollar value for revenue of the company in millions| Companies are ranked through this attribute|float64|
|Profit             | Dollar value for profit of the company in millions| |float64|
|Number of Employees| Number of people the company has employed| |int64|
|Sector             | The sector of the market the company operates in| |object|
|City               | The city where the company has its headquarters | All the companies are in the United Stated |object|
|State              | The state where the company has its headquarters | All the companies are in the United Stated |object|
|Newcomer           | Indicates whether the company is new to the top Fortune 500|  No value assigned to companies outside of the top 500|object (yes/no)|
|CEO Founder        | Indicates whether the current CEO of the company is also its founder| |object (yes/no)|
|CEO Woman          | Indicates whether the CEO of the company is a woman | | object (yes/no)|
|Profitable         | Indicates whether the company has profited over the last year or not| |object (yes/no)|
|Previous Rank      | The ranking given to the company by Fortune in 2020| No value assigned to companies outside of the top 500|float64|
|CEO                | The name of the CEO of the company| |object|
|Website            | The url to the company's website| |object|
|Ticker             | The stock ticker symbol of the company| Values assigned only for public companies, value is null if the company is privately-owned|object|
|Market Cap         | The total dollar market value of the company's outstanding shares of stock in millions| Values null in the case of some private companies|float64|

With this dataset, the aim is to provide the viewer with some general details about top U.S. *corporations*, like the *sector in which it operates*, *headquarter locations*, *CEO information*, *company website*, *stock market ticker*, and explore the statistics, like their *revenue*, *yearly profit*, *employee count* and *market cap*, while using data pulled exclusively from Fortune, like the *ranking* for individual corporations. These features help in providing answers to questions like,
- *Which companies constitute the top 10% of the world's biggest companies?*
- *Does profitability play a role in ranking?*
- *Which sector is most popular in the market?*
- *Are there places (hotspots) where many companies fix their base of operations at?*
- *What percentage of companies have CEOs who were also founders?*
- *What proportion of companies have women CEOs?*
- *What proportion of companies are privately-owned?*

In addition to this, key factors regarding the spread and centrality shall be discussed, followed by a systematic approach to clean the dataset by resolving inconsistencies, removing missing values and replacing outliers.




## Data Cleaning 
1. Importing Pandas and Matplotlib
2. Loading the data 
3. Rearrange the data
4. Drop irrelevant attributes
5. Detect and remove rows with missing data
6. Detect outliers

### Importing Pandas and Matplotlib
- When using Jupyter Notebook with a dark editor the axes, tick, text etc. are difficult with a black theme to see since they are black by default so I will change them to gray
- Adjust font size
- Import Pandas and Matplotlib

In [ ]:
import matplotlib as mpl

font = {'family' : 'Georgia',
        'weight' : 'bold',
        'size'   : 12}

COLOR = 'gray'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

mpl.rc('font', **font)

import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # Suppress scientific notation for all floats 

### Retrieving the dataset and sorting by rank

In [ ]:
df = pd.read_csv("Fortune_1000.csv", index_col=['rank']).sort_index()
df.head(10)

### Rearranging the data

The columns that we plan to work with frequently have been moved to the left.

In [ ]:
df = df[['company', 'revenue', 'profit', 'Market Cap', 'num. of employees', 'sector', 'city', 'state', 'CEO', 'prev_rank', 'rank_change', 'newcomer', 'ceo_founder', 'ceo_woman', 'profitable', 'Ticker', 'Website']]
df.head()

<br>Some information pertaining to the dataset and its attributes,<br>

In [ ]:
df.info()

<br>Some attributes like Market Cap and prev_rank have object as their dtype instead of int64 and float64, due to presence of null values or spaces (instead of leaving the field blank).
This can be resolved easily,<br>

In [ ]:
df[['Market Cap', 'prev_rank']] = df[['Market Cap', 'prev_rank']].apply(pd.to_numeric, errors='coerce')
df.info()

### Removing Attributes

Moreover, some attributes are not needed for this study since they have too many values missing by virtue of their definition or are not relevant, and can be dropped including:
- website 
- prev_rank
- rank_change 
- newcomer 

In [ ]:
df = df.drop(columns=['prev_rank', 'rank_change', 'newcomer', 'Website'])
df.info()

### Detect and remove rows with missing data

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()
df.isna().sum()

### Detect and remove outliers
- The method chosen to remove outliers will depend on the density or distribution of each column. To get a sense of the distribution I will:
    - Display some basic statistical info about the columns
    - Plot the density or distribution of the numeric columns
    - Plot box plot to see your distribution and potential outliers. (Optional)

### Descriptive statistics

Attributes like revenue, profit and num. of employees play a key role in differentiating a company's ranking in terms of valuation and popularity. Some measures of spread/central tendency for these key features are listed below,

In [ ]:

categorical_attributes = ['sector','city','state','newcomer','ceo_founder','ceo_woman','profitable']
numerical_attributes = ["revenue","profit","Market Cap","num. of employees"]
# target_attribute = ["output"]
stats = ['mean', 'median', 'std', 'var']
agg_dict = {'revenue': stats, 'profit': stats, 'Market Cap': stats, 'num. of employees': stats}
desc_stats = df.agg(agg_dict).round(3)
desc_stats


In [ ]:
df.describe()

From this summary, the standard deviations look huge, so this would mean the data must be skewed and greatly spread apart, with a larger number of outliers. Below are the histograms for the above attributes,

In [ ]:
def plot_histograms_density(df, columns):
    temp = df.copy()
    fig, axs = plt.subplots(len(columns), 1, figsize = (20,20))
    fig.tight_layout(pad=5.0)
    i = 0
    for c in columns:
        temp[c].hist(ax = axs[i], bins=100) # normalizes the density
        temp[c].plot.density(ax = axs[i], title = c)
        i += 1

plot_histograms_density(df, numerical_attributes)

Plotting the distribution shows that most numerical data is positively skewed so IQR will be used to handle outliers 

### Detecting the outliers 
- The *lower limit* for each numerical column is $Q1 - 1.5 * IQR$ 
- The *upper limit* for each numerical column is $Q3 + 1.5 * IQR$ 
- Values less than the *lower limit* will be set to *lower limit*
- Values less than the *upper limit* will be set to *lower limit*
- Essentially each column is capped to [*lower limit*, *upper limit*]

In [ ]:
def show_outliers_iqr(df, columns, verbose):
    temp = df.copy()
    total_outliers = 0
    total_data = 0
    for col in columns:
        Q1 = temp[col].quantile(0.25)
        Q3 = temp[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR
    
        column_outliers = len(temp[temp[col] > upper_limit]) + len(temp[temp[col] < lower_limit]) # Outliers in this column
        total_outliers += column_outliers # Add the total outliers in this column to the running total of outliers
        total_data += temp[col].shape[0]    # Total rows in this column
        if (verbose):
            print(f"Column: {col}")
            print(f"The minimum and maximum extremes are: [{round(lower_limit, 2)}, {round(upper_limit, 2)}]")
            print(f"Outliers: {column_outliers}")
            print(f"Outliers in this column: {round(column_outliers/temp[col].shape[0], 2)} % \n")
    print(f"Total outliers for dataframe: {round(total_outliers / total_data * 100, 2)} %")

show_outliers_iqr(df, numerical_attributes, True) # Verbose can be set to false to stop seeing outlier data for each column

 

## Charts

Some additional charts have been plotted for better understanding of the data and for providing hints to questions put forth at the beginning,

In [ ]:
fig, axs = plt.subplots(2, 2)
axs = axs.flatten()

df['sector'].value_counts(normalize=True).plot.pie(ax=axs[0], 
                                                   figsize=(12,10), autopct='%.1f %%', 
                                                   colormap='tab20', labels=None, title='Proportions of sectors companies work in ↓',
                                                   legend=True, ylabel='', fontsize=8, 
                                                   radius=1.2).legend(labels=df['sector'], bbox_to_anchor=(-0.6,1.1), loc='upper left')
df['profitable'].value_counts(normalize=True).plot.pie(ax=axs[1], 
                                                       figsize=(12,10), radius=1.1, 
                                                       autopct='%.1f %%', colormap='tab10',
                                                       title='Proportion of profitable companies ↓', 
                                                       legend=True, ylabel='', fontsize=12)
df['ceo_founder'].value_counts(normalize=True).plot.pie(ax=axs[2], 
                                                       figsize=(12,10), radius=1.1, 
                                                       autopct='%.1f %%', colormap='tab10',
                                                       title='Proportion of CEOs who are also founders ↓', 
                                                       legend=True, ylabel='', fontsize=12)
df['ceo_woman'].value_counts(normalize=True).plot.pie(ax=axs[3], 
                                                       figsize=(12,10), radius=1.1, 
                                                       autopct='%.1f %%', colormap='tab10',
                                                       title='Proportion of women CEOs ↓', 
                                                       legend=True, ylabel='', fontsize=12)

plt.show()

From these charts, it can be seen that a greater portion of companies are invested into Retail, Energy, Technology and Healthcare sectors. But most companies remain profitable, whatever sector of the market they may be operating in. 

Also, the there only a few amount of companies having CEOs that are women or founded the company themselves; this may point to possible bias in hiring/selecting CEOs, or simply a shortage of women candidates applying for CEO positions, while the fact that there aren't many CEOs who are founders too is not surprising, as most big corporations have been around for awhile, and can potentially see many CEO successors. 

<br>Plotted below is the horizontal histogram for the frequency of company headquarters per state in the U.S.,

In [ ]:
# making a plot showing the frequency of companies headquartered in each U.S. state

def barPlot(series, x_lab, y_lab, title):
    series.value_counts().plot.barh(
                                figsize=(20, 10), 
                                width=1, 
                                edgecolor='black', 
                                xlabel=x_lab,
                                ylabel=y_lab, 
                                title=title, 
                                fontsize=12,
                                )
    
barPlot(df['state'], 'Number of Companies', 'Headquarters', 'Headquarter Frequency per U.S. State')

As the plot demonstrates, some hotspots for companies include California, Texas and New York.